<a href="https://colab.research.google.com/github/jespimentel/sis_geo/blob/main/plotador_jecrim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

import folium
from folium.plugins import HeatMap, TagFilterButton

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_1 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_0_2000.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_2001_3314.csv')
df_partes = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/partes.csv')
df = pd.concat([df_1,df_2] , ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Número TJ             3168 non-null   object 
 1   Número MP             3168 non-null   object 
 2   Tipo de Procedimento  3168 non-null   object 
 3   Situação              3168 non-null   object 
 4   Delegacia             3164 non-null   object 
 5   Assunto               3168 non-null   object 
 6   DtFatoInicial         3168 non-null   object 
 7   Violência Doméstica   3167 non-null   object 
 8   Município             3168 non-null   object 
 9   Logradouro            3168 non-null   object 
 10  Num_Logradouro        2904 non-null   object 
 11  Bairro                3019 non-null   object 
 12  Latitude              3168 non-null   float64
 13  Longitude             3168 non-null   float64
dtypes: float64(2), object(12)
memory usage: 346.6+ KB


In [6]:
df['Tipo de Procedimento'].value_counts()

Inquérito Policial - IP             2357
Medida Cautelar MC                   727
Termo Circunstanciado - TCO           44
Auto de Prisão em Flagrante           38
Inquérito Policial Militar - IPM       2
Name: Tipo de Procedimento, dtype: int64

In [7]:
df_partes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Número MP  3592 non-null   object
 1   Partes     3592 non-null   object
dtypes: object(2)
memory usage: 56.2+ KB


In [9]:
# JECRIM
criterio = (df['Tipo de Procedimento'] == 'Termo Circunstanciado - TCO')
df_jecrim = df.loc[criterio][['Número TJ', 'Assunto', 'DtFatoInicial','Latitude','Longitude']]
df_jecrim['Assunto'] = df_jecrim['Assunto'].apply(lambda x: x.strip())
df_jecrim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 263 to 3148
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Número TJ      44 non-null     object 
 1   Assunto        44 non-null     object 
 2   DtFatoInicial  44 non-null     object 
 3   Latitude       44 non-null     float64
 4   Longitude      44 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.1+ KB


In [11]:
# Converter a coluna 'DtFatoInicial' para o tipo datetime
df_jecrim['DtFatoInicial'] = pd.to_datetime(df_jecrim['DtFatoInicial'], dayfirst=False)
df_jecrim['DiaDaSemana'] = df_jecrim['DtFatoInicial'].dt.day_name()
df_jecrim['DtFatoInicial'] = df_jecrim['DtFatoInicial'].dt.strftime('%d/%m/%Y')

traducao_dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df_jecrim['DiaDaSemana'] = df_jecrim['DiaDaSemana'].map(traducao_dias_semana)

df_jecrim.head(5)

<ipython-input-11-005a9e944834>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_jecrim['DtFatoInicial'] = pd.to_datetime(df_jecrim['DtFatoInicial'], dayfirst=False)


,Número TJ,Assunto,DtFatoInicial,Latitude,Longitude,DiaDaSemana
263,1500620-40.2023.8.26.0451,Ameaça,02/11/2023,-22.72562,-47.64630,Quinta-feira
494,1547175-52.2022.8.26.0451,Desobediência,31/12/2022,-22.72618,-47.64958,Sábado
669,1532456-65.2022.8.26.0451,Ameaça,12/04/2022,-22.75051,-47.64127,Terça-feira
696,1532437-59.2022.8.26.0451,Despenalização / Descriminalização,30/11/2022,-22.74488,-47.64920,Quarta-feira
787,1532279-04.2022.8.26.0451,Jogo de azar,14/11/2022,-22.67081,-47.69342,Segunda-feira


In [12]:
assuntos = list(set(df_jecrim['Assunto'].tolist()))
dias_da_semana = list(set(df_jecrim['DiaDaSemana'].tolist()))
assuntos

['Receptação culposa',
 'Ato Obsceno',
 'Praticar lesão corporal culposa na direção de veículo automotor',
 'Adulteração de Sinal Identificador de Veículo Automotor',
 'Despenalização / Descriminalização',
 'parágrafo único - Aumenta-se a pena de um terço à metade, se ocorrer qualquer das hipóteses do parágrafo único do artigo anterior',
 'Injúria',
 'Conduzir veículo automotor, na via pública, estando com concentração de  álcool por litro de sangue igual ou superior a 6 (seis) decigramas, ou sob a influência de qualquer outra substância psicoativa que determine dependência',
 'Jogo de azar',
 'Desacato',
 'Desobediência',
 'Ameaça']

In [13]:
import folium
import random

# Dicionário para mapear cores aos assuntos
cores_assuntos = {}
cores_predefinidas = ['darkgreen', 'pink', 'lightgreen', 'darkpurple', 'lightgray', 'gray',
                      'beige', 'red', 'lightred', 'blue', 'cadetblue', 'darkred', 'black',
                      'purple', 'lightblue', 'white', 'green', 'darkblue', 'orange']
for assunto in assuntos:
    cor_assunto = random.choice(cores_predefinidas)
    cores_assuntos[assunto] = cor_assunto

PIRA_COORDINATES = (-22.769960, -47.647579)

# Cria um mapa vazio em Piracicaba
_map = folium.Map(location=PIRA_COORDINATES, zoom_start=12)

# Adiciona um título ao mapa
title_html = """
<h1 style="text-align:center;"><b>DDM - TCOs para data do fato entre 01/01/2022 e 24/04/2023</b></h1>
<h2 style="text-align:center;"><b>Base: SISMP INTEGRADO - Extração em 24/04/2023</b></h2>
"""
_map.get_root().html.add_child(folium.Element(title_html))

# Adiciona o marcador para cada ocorrência
for each in df_jecrim.iterrows():
    assunto = each[1]['Assunto']
    dia_da_semana = each[1]['DiaDaSemana']
    cor_assunto = cores_assuntos[assunto]
    folium.Marker(
        location=[each[1]["Latitude"], each[1]["Longitude"]],
        clustered_marker=True,
        tags=[assunto, dia_da_semana],
        popup=f"{assunto}<br>{each[1]['DtFatoInicial']}<br>{each[1]['Número TJ']}<br>{each[1]['DiaDaSemana']}",
        icon=folium.Icon(color=cor_assunto)
    ).add_to(_map)

# Adiciona botões de filtro por tag
TagFilterButton(assuntos, clear_text='Todos', name='Assuntos').add_to(_map)
TagFilterButton(dias_da_semana, clear_text="Todos",name='DiaDaSemana').add_to(_map)

display(_map)
_map.save('violencia_domestica.html')